Prediksi Penyakit Liver Dengan Jaringan Syaraf Tiruan

In [1]:
#Import Library
import pandas as pd
import numpy as np

#Membaca Data
df = pd.read_csv("C:/Users/USER/Documents/Python/Jaringan Syaraf Tiruan Project/indian_liver_patient.csv")

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         583 non-null    int64  
 1   Gender                      583 non-null    object 
 2   Total_Bilirubin             583 non-null    float64
 3   Direct_Bilirubin            583 non-null    float64
 4   Alkaline_Phosphotase        583 non-null    int64  
 5   Alamine_Aminotransferase    583 non-null    int64  
 6   Aspartate_Aminotransferase  583 non-null    int64  
 7   Total_Protiens              583 non-null    float64
 8   Albumin                     583 non-null    float64
 9   Albumin_and_Globulin_Ratio  579 non-null    float64
 10  Dataset                     583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB
None


In [2]:
#Check for missing values
print(df.isnull().sum())

print(df['Albumin_and_Globulin_Ratio'].describe())

Age                           0
Gender                        0
Total_Bilirubin               0
Direct_Bilirubin              0
Alkaline_Phosphotase          0
Alamine_Aminotransferase      0
Aspartate_Aminotransferase    0
Total_Protiens                0
Albumin                       0
Albumin_and_Globulin_Ratio    4
Dataset                       0
dtype: int64
count    579.000000
mean       0.947064
std        0.319592
min        0.300000
25%        0.700000
50%        0.930000
75%        1.100000
max        2.800000
Name: Albumin_and_Globulin_Ratio, dtype: float64


In [3]:
#Mengisi missing values dengan median
df['Albumin_and_Globulin_Ratio'].fillna(df['Albumin_and_Globulin_Ratio'].median(), inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         583 non-null    int64  
 1   Gender                      583 non-null    object 
 2   Total_Bilirubin             583 non-null    float64
 3   Direct_Bilirubin            583 non-null    float64
 4   Alkaline_Phosphotase        583 non-null    int64  
 5   Alamine_Aminotransferase    583 non-null    int64  
 6   Aspartate_Aminotransferase  583 non-null    int64  
 7   Total_Protiens              583 non-null    float64
 8   Albumin                     583 non-null    float64
 9   Albumin_and_Globulin_Ratio  583 non-null    float64
 10  Dataset                     583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB
None


C:\Users\USER\AppData\Local\Temp\ipykernel_16180\2480636606.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Albumin_and_Globulin_Ratio'].fillna(df['Albumin_and_Globulin_Ratio'].median(), inplace=True)


In [4]:
# Encode categorical 'Gender' column
df_encoded = df.copy()
df_encoded['Gender'] = df_encoded['Gender'].map({'Male': 1, 'Female': 0})

# Separate features and target
X = df_encoded.drop('Dataset', axis=1).values
y = df_encoded['Dataset'].values

# Normalize features
X_norm = (X - X.mean(axis=0)) / X.std(axis=0)

# Split data into features and target
def custom_train_test_split(X, y, test_size=0.2):
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    split = int((1 - test_size) * len(indices))
    return X[indices[:split]], X[indices[split:]], y[indices[:split]], y[indices[split:]]

X_train, X_test, y_train, y_test = custom_train_test_split(X_norm, y)


In [13]:
import numpy as np

# Fungsi aktivasi sigmoid dan turunannya
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Fungsi training model neural network
def train_backprop(X_train, y_train, hidden_size=10, lr=0.1, epochs=10000):
    input_size = X_train.shape[1]
    output_size = 1

    # Inisialisasi bobot dan bias secara acak
    np.random.seed(42)
    weights_input_hidden = np.random.rand(input_size, hidden_size)
    bias_hidden = np.random.rand(1, hidden_size)
    weights_hidden_output = np.random.rand(hidden_size, output_size)
    bias_output = np.random.rand(1, output_size)

    for epoch in range(epochs):
        # Forward pass
        hidden_input = np.dot(X_train, weights_input_hidden) + bias_hidden
        hidden_output = sigmoid(hidden_input)

        final_input = np.dot(hidden_output, weights_hidden_output) + bias_output
        final_output = sigmoid(final_input)

        # Hitung loss (MSE)
        loss = np.mean((y_train - final_output) ** 2)

        # Backpropagation
        error_output = y_train - final_output
        d_output = error_output * sigmoid_derivative(final_output)

        error_hidden = d_output.dot(weights_hidden_output.T)
        d_hidden = error_hidden * sigmoid_derivative(hidden_output)

        # Update bobot dan bias
        weights_hidden_output += hidden_output.T.dot(d_output) * lr
        bias_output += np.sum(d_output, axis=0, keepdims=True) * lr

        weights_input_hidden += X_train.T.dot(d_hidden) * lr
        bias_hidden += np.sum(d_hidden, axis=0, keepdims=True) * lr

        # Cetak loss tiap 1000 epoch
        if epoch % 1000 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}")

    # Kembalikan model terlatih
    return {
        'weights_input_hidden': weights_input_hidden,
        'bias_hidden': bias_hidden,
        'weights_hidden_output': weights_hidden_output,
        'bias_output': bias_output
    }

# Fungsi prediksi
def predict(X, model):
    hidden_input = np.dot(X, model['weights_input_hidden']) + model['bias_hidden']
    hidden_output = sigmoid(hidden_input)

    final_input = np.dot(hidden_output, model['weights_hidden_output']) + model['bias_output']
    final_output = sigmoid(final_input)

    return (final_output > 0.5).astype(int)
# Train the model
model = train_backprop(X_train, y_train.reshape(-1, 1))

# Predict on test data and print accuracy
y_pred = predict(X_test, model)
accuracy = np.mean(y_pred.flatten() == y_test)
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 0, Loss: 0.2969
Epoch 1000, Loss: 0.2640
Epoch 2000, Loss: 0.2640
Epoch 3000, Loss: 0.2640
Epoch 4000, Loss: 0.2640
Epoch 5000, Loss: 0.2639
Epoch 6000, Loss: 0.2639
Epoch 7000, Loss: 0.2639
Epoch 8000, Loss: 0.2639
Epoch 9000, Loss: 0.2639
Test Accuracy: 0.6239


In [ ]:
pasien_baru = np.array([[65, 1, 0.7, 0.1, 187, 16, 18, 6.8, 3.3, 0]])

# Normalisasi pasien_baru menggunakan mean dan std dari X
pasien_baru_scaled = (pasien_baru - X.mean(axis=0)) / X.std(axis=0)

# Prediksi menggunakan model backpropagation
hasil_prediksi = predict(pasien_baru_scaled, model)

# Interpretasi hasil prediksi
print("\n=== Hasil Prediksi Pasien Baru ===")
if hasil_prediksi[0][0] == 2:
    print("Pasien kemungkinan MENDERITA penyakit liver.")
else:
    print("Pasien kemungkinan TIDAK menderita penyakit liver.")


=== Hasil Prediksi Pasien Baru ===
Pasien kemungkinan TIDAK menderita penyakit liver.


In [ ]:
pasien_baru_scaled = (pasien_baru - X.mean(axis=0)) / X.std(axis=0)

# Prediksi menggunakan model backpropagation
hasil_prediksi = predict(pasien_baru_scaled, model)

# Interpretasi hasil prediksi
print("\n=== Hasil Prediksi Pasien Baru ===")
if hasil_prediksi[0][0] == 2:
    print("Pasien kemungkinan MENDERITA penyakit liver.")
else:
    print("Pasien kemungkinan TIDAK menderita penyakit liver.")